In [48]:
import tensorflow as tf
import keras
from keras.optimizers import*
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import*
from keras.initializers import*
from keras.models import Model
#from keras.utils.training_utils import multi_gpu_model   #导入keras多GPU函数
import matplotlib.pyplot as plt
import numpy as np
#import pandas as pd

In [49]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.savefig('loss.png')
        plt.show()


In [50]:
x_train=np.load("../input/2048data/data/x_train.npy")
x_test=np.load("../input/2048data/data/x_test.npy")
x_val=np.load("../input/2048data/data/x_val.npy")
y_train=np.load("../input/2048data/data/y_train.npy")
y_test=np.load("../input/2048data/data/y_test.npy")
y_val=np.load("../input/2048data/data/y_val.npy")
val=(x_val,y_val)

In [51]:
def identity_block(X, f, filters):
 
 
    F1, F2, F3 = filters
 
    X_shortcut = X
 
    X = Conv2D(filters = F1, kernel_size = (1,1), strides = (1,1), padding = 'valid', kernel_initializer = glorot_uniform(seed = 0))(X)#xavier initializer
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
 
    X = Conv2D(filters = F2, kernel_size = (f,f), strides = (1,1), padding = 'same', kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
 
    X = Conv2D(filters = F3, kernel_size = (1,1), strides = (1,1), padding = 'valid', kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3)(X)
 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
 
    return X

In [52]:
def convolution_block(X, f, filters, s=1):
 

    F1, F2, F3 = filters
 
    X_shortcut = X
 
    X = Conv2D(filters = F1, kernel_size = (1,1), strides = (s,s), kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
 
    X = Conv2D(filters = F2, kernel_size = (f,f), strides = (1,1), padding = 'same', kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
 
    X = Conv2D(filters = F3, kernel_size = (1,1), strides = (1,1), kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3)(X)
 
    X_shortcut = Conv2D(F3, (1,1), strides = (s,s), kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut)
 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
 
    return X

In [53]:
def my_ResNet(input_shape = (4,4,11), classes = 4):
 
    X_input = Input(input_shape)
 
    X = ZeroPadding2D((2,2))(X_input)
 
    X = Conv2D(64, (2, 2), strides = (1,1),padding='same', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides = (2,2))(X)
 
    X = convolution_block(X, f = 3, filters = [64,64,256])
    X = identity_block(X, 3, [64,64,256])
    X = identity_block(X, 3, [64,64,256])
 
    X = convolution_block(X, f = 3, filters = [128,128,512])
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])
    X = identity_block(X, 3, [128,128,512])
 
    X = convolution_block(X, f = 3, filters = [256,256,1024])
    X = identity_block(X, 3, [256,256,1024])
    X = identity_block(X, 3, [256,256,1024])
    X = identity_block(X, 3, [256,256,1024])    
    X = identity_block(X, 3, [256,256,1024])
    X = identity_block(X, 3, [256,256,1024])
 
    X = convolution_block(X, f = 3, filters = [512,512,2048])
    X = identity_block(X, 3, [512,512,2048])
    X = identity_block(X, 3, [512,512,2048])
 
    X = AveragePooling2D((2, 2))(X)
 
    X = Flatten()(X)
    X = Dense(classes, activation = 'softmax', kernel_initializer = glorot_uniform(seed=0))(X)
 
    model = Model(inputs = X_input, outputs = X, name = 'my_ResNet')
 
    return model

In [55]:
model=my_ResNet()
adadelta=Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy',optimizer='Adadelta',metrics=['accuracy'])
history=LossHistory()
model.fit(x=x_train, y=y_train, batch_size=250, epochs=2, verbose=1,callbacks=[history],validation_data=val)  
                         
model.save_weights("testmodel.h5")
print("model save success!")
# show_train_history(train_history,'acc','val_acc')
# show_train_history(train_history, 'loss', 'val_loss')
scores = model.evaluate(x_test,
                        y_test,
                        verbose=0)
history.loss_plot('epoch')
print('accuracy=', scores[1])

Epoch 1/2


KeyboardInterrupt: 